In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [15]:
import time
import random
import nltk
import string
from nltk.corpus import words

In [4]:
nltk.download('words')

[nltk_data] Downloading package words to /home/dkc/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [5]:
english_words = words.words()
length_5_words= []
length_5_non_repeating_char_words = []
for el in english_words:
    if len(el)==5:
        length_5_words.append(el.upper())
        if len(set(el.upper()))==5:
            length_5_non_repeating_char_words.append(el.upper())
            
random.shuffle(length_5_non_repeating_char_words)

In [6]:
class MyList:
    def __init__(self):
        self.elements =[]

    def insert(self, element):
        self.elements.append(element)

    def search(self, data):
        return data in self.elements

# Students have to write similar classes for MySortedList, MyBST and MyHashMap

In [7]:
def give_colors(true_word, guess):
    """ takes two strings of length N. 
    returns a list of length N with values 0,1 or 2. 
    No repeating characters in both true word and guess.
    """
    ans = []
    N = len(guess)
    for i in range(N):
        if guess[i] not in true_word:
            ans.append(0)
        elif guess[i] == true_word[i]:
            ans.append(2)
        else:
            ans.append(1)
    return ans

In [12]:
def is_candidate(guess_list,colors_list, word_to_check):
    for i in range(len(guess_list)):
        colors = give_colors(true_word=word_to_check, guess=guess_list[i])
        if colors != colors_list[i]:
            return False
    return True

In [36]:
def guess_word_from_prev(guess_list,colors_list,simple_word_list, num_try):
    # Students have to write this function also.

    # guess_list : list of length k,  (integer k, can be zero)
    # each element in this list is a string of length N. 
    #  string only contains upper case A to Z
     
    #  colors_list : list of length k,
    #  each element in this list is a list of length N.
    #  each element in this list is either 0 or 1 or 2.
    #  0 stands for gray, 1 for yellow and 2 for green.

    #  simple_word_list is a list of length V, containing 
    #  a small subset of the set of words in the vocab list.
    #  each element in this list is a string of length N. 
    #  string only contains upper case A to Z.
    #  Can be used for initial few guesses.

    #  num_try is a number indicating how many times we have failed in generating a
    #  valid vocabulary word with the current guess/color info
    
    #  Returns a guess of the possible answer. 
    #  This need not be a word in the vocabulary 
    #  as you do not have access to the vocabulary here.

    #return simple_word_list[random.randint(0,len(simple_word_list)-1)] 
    # example dumb guess which always guesses a random simple word. 
    # Note that this is guaranteed to return a valid word, but will NEVER get the right answer if the true solution is not a simple word
    if len(guess_list) == 0:
        return simple_word_list[random.randint(0, len(simple_word_list)-1)]
    elif len(guess_list) <= 4 or num_try > 100:
        new_simple_word_list = [word for word in simple_word_list if word not in guess_list]
        for word in simple_word_list:
            if is_candidate(guess_list,colors_list, word):
                new_simple_word_list.append(word)
        if new_simple_word_list:
            return new_simple_word_list[random.randint(0, len(new_simple_word_list)-1)]
        else:
            return simple_word_list[random.randint(0, len(simple_word_list)-1)]
    else:
        # Define the pool of characters
        letters = list(string.ascii_uppercase)
        #remove letters which are 0s in colors_list
        for i in range(len(guess_list)):
            for j in range(len(simple_word_list[0])):
                if colors_list[i][j] == 0 and guess_list[i][j] in letters:
                    letters.remove(guess_list[i][j])
        random_word = ''.join(random.choices(letters, k=len(simple_word_list[0])))
        return random_word
        # while 2>1:
        #     random_word = ''.join(random.choices(letters, k=len(simple_word_list[0])))
        #     if is_candidate(guess_list,colors_list, random_word):
        #         return random_word
            



In [37]:
# Tournament 1
# Students are not expected to write/modify anything here, and is only meant to show how their 
# guess_word_from_prev function, and the insert/search functions in their implementations 
# of MySortedList, MyBST and MyHashMap will be used. 
# Using any variable names here as a hint in the functions to be implemented is 
# illegal and will backfire, as all variable names in this cell can be changed.

legal_word_list = length_5_non_repeating_char_words[:-2000] # all but the last 2000 words
simple_word_list = legal_word_list[:100] # first 100 words as a starter pack that is given.

N = len(simple_word_list[0]) # number of characters in each word
start = time.time()
vocabulary = MyList() # It can also be MyBST or MySortedList or MyHashMap
# all of these classes must support an insert function and a search function
for el in legal_word_list:
    vocabulary.insert(el) 
wordle_answers = []
for j in range(20): # 20 puzzles per tournament
    wordle_answers.append(legal_word_list[random.randint(0,len(legal_word_list)-1)])
    wordle_answers.append(simple_word_list[random.randint(0,len(simple_word_list)-1)])

for j in range(20):
    correct_solution = wordle_answers[j]
    guess_list = []
    colors_list = []

    while 2>1: # In each cycle of this loop one valid guess is made and a new colors info is acquired.
        num_try = 0 # try number to guess a word in vocabulary that agrees with the results so far
        error_flag = False
        while 2>1: 
            # Within each cycle of this loop an attmept to find a word in the vocabulary that obeys colors is made.
            # the loop terminates when a valid word in the vocabulary is found
            guess =  guess_word_from_prev(guess_list, colors_list, simple_word_list, num_try)
            num_try += 1
            if vocabulary.search(guess):
                break
            if num_try > 10000:
                error_flag = True
                break
        if error_flag:
            break
        guess_list.append(guess)
        colors = give_colors(correct_solution, guess)
        colors_list.append(colors)
        if sum(colors) == 2*N:
            break
        if len(guess_list)>1000:
            error_flag = True
            break
    print("Tournament 1, puzzle ",j)
    if error_flag:
        print("Tried too many non-dictionary words in a single situation or tried too many guesses.")
    else:
        print("Guesses :", guess_list)
        print("Number of guesses :", len(guess_list))
        print("Solution :", correct_solution)
    print("====================")

end = time.time()
print(f'total time taken is {round(end-start,4)} secs')
            
        









Tournament 1, puzzle  0
Guesses : ['AWHET', 'SCRIN', 'SPIEL', 'FODGE', 'DALER', 'KITAR', 'YOKEL', 'CARIB', 'SPYER', 'WRUNG', 'COXAL', 'MAUVE', 'ROUND', 'FUNDS', 'MUSHY', 'TOPAZ', 'ALIKE', 'MARSI', 'URBAN', 'WHALY', 'AOIFE', 'SIMON', 'DUNGY', 'NAZIM', 'FRIZE', 'BAKIE', 'WAGES', 'DUTCH', 'CHALK', 'SMOUT', 'DRAWK', 'PIMAN', 'CAGIT', 'SNUCK', 'SOMAL', 'TAICH', 'CHANE', 'INLET', 'MAIDY', 'LEACH', 'ROHUN', 'OREAD', 'SLANG', 'HARPY', 'TENCH', 'ISLAM', 'LOCKY', 'HAJIB', 'VIBEX', 'TANGS', 'SIDTH', 'RAISE', 'BENJY', 'ADNEX', 'ENAPT', 'COLZA', 'NOTER', 'MANTO', 'AKEBI', 'VARDY', 'BERYL', 'CLOUD', 'TAPEN', 'PUIST', 'GROIN', 'RUMAL', 'PERCH', 'TRONE', 'RAMED', 'CUBIT', 'CEBID', 'ACOLD', 'BEGAT', 'SPENT', 'NITRO', 'INERT', 'DETUR', 'CLEFT', 'YIRTH', 'REMOP', 'MOSTE', 'ODIUM', 'PARTY', 'SERUT', 'COPRA', 'REBUS', 'TAIPO', 'KLING', 'SNURP', 'STONE', 'ENOIL', 'RATIO', 'BODER', 'JATNI', 'CRUTH', 'ORANG', 'BASTE', 'LONGS', 'PYGAL', 'SAUTE', 'ROAST', 'ROHUN', 'TAICH', 'FRIZE', 'SNUCK', 'WAGES', 'HAJIB', 'N

In [31]:
# creating the BST DS
class Node:
    def __init__(self, data):
        self.val = data
        self.left = None
        self.right = None
        self.parent = None

In [32]:
class MyBST:
    def __init__(self):
        self.root = None

    def insert(self, element):
        if self.root is None:
            self.root = Node(element)
            return
        else: 
            curr = self.root
            new_node = Node(element)
            while curr is not None:
                if new_node.val == curr.val:
                    return
                if new_node.val > curr.val:
                    if curr.right is None:
                        curr.right = new_node
                        return
                    else:
                        curr = curr.right
                elif new_node.val < curr.val:
                    if curr.left is None:
                        curr.left = new_node
                        return
                    else:
                        curr = curr.left                

    def search(self, data):
        curr = self.root
        while curr is not None:
            if curr.val == data:
                return True
            elif curr.val > data:
                curr = curr.left
            else:
                curr = curr.right
        return False

In [ ]:
# Tournament 1
# Students are not expected to write/modify anything here, and is only meant to show how their 
# guess_word_from_prev function, and the insert/search functions in their implementations 
# of MySortedList, MyBST and MyHashMap will be used. 
# Using any variable names here as a hint in the functions to be implemented is 
# illegal and will backfire, as all variable names in this cell can be changed.

legal_word_list = length_5_non_repeating_char_words[:-2000] # all but the last 2000 words
simple_word_list = legal_word_list[:100] # first 100 words as a starter pack that is given.

N = len(simple_word_list[0]) # number of characters in each word
start = time.time
vocabulary = MyBST() # It can also be MyBST or MySortedList or MyHashMap
# all of these classes must support an insert function and a search function
for el in legal_word_list:
    vocabulary.insert(el) 
wordle_answers = []
for j in range(20): # 20 puzzles per tournament
    wordle_answers.append(legal_word_list[random.randint(0,len(legal_word_list)-1)])
    wordle_answers.append(simple_word_list[random.randint(0,len(simple_word_list)-1)])

for j in range(20):
    correct_solution = wordle_answers[j]
    guess_list = []
    colors_list = []

    while 2>1: # In each cycle of this loop one valid guess is made and a new colors info is acquired.
        num_try = 0 # try number to guess a word in vocabulary that agrees with the results so far
        error_flag = False
        while 2>1: 
            # Within each cycle of this loop an attmept to find a word in the vocabulary that obeys colors is made.
            # the loop terminates when a valid word in the vocabulary is found
            guess =  guess_word_from_prev(guess_list, colors_list, simple_word_list, num_try)
            num_try += 1
            if vocabulary.search(guess):
                break
            if num_try > 10000:
                error_flag = True
                break
        if error_flag:
            break
        guess_list.append(guess)
        colors = give_colors(correct_solution, guess)
        colors_list.append(colors)
        if sum(colors) == 2*N:
            break
        if len(guess_list)>1000:
            error_flag = True
            break
    print("Tournament 1, puzzle ",j)
    if error_flag:
        print("Tried too many non-dictionary words in a single situation or tried too many guesses.")
    else:
        print("Guesses :", guess_list)
        print("Number of guesses :", len(guess_list))
        print("Solution :", correct_solution)
    print("====================")

end = time.time
print(f'total time taken is {round(end-start,4)} secs')

In [ ]:
#creating the sorted list DS
class MySortedList:
    def __init__(self):
        self.elements =[]

    def insert(self, element):
        #self.elements.append(element)
        #self.elements.sort()
        left = 0
        right = len(self.elements)-1
        while left <= right:
            mid = (left + right) //2
            if element > self.elements[mid]:
                left = mid + 1
            else:
                right = mid -1
        self.elements.insert(left, element)

    def search(self, data):
        left = 0
        right = len(self.elements) - 1
        while (left <= right):
            mid = (left + right) //2
            candidate = self.elements[mid]
            if candidate == data:
                return True
            elif candidate > data:
                right = mid -1
            elif candidate < data:
                left = mid+1
        return False

In [ ]:
# Tournament 1
# Students are not expected to write/modify anything here, and is only meant to show how their 
# guess_word_from_prev function, and the insert/search functions in their implementations 
# of MySortedList, MyBST and MyHashMap will be used. 
# Using any variable names here as a hint in the functions to be implemented is 
# illegal and will backfire, as all variable names in this cell can be changed.

legal_word_list = length_5_non_repeating_char_words[:-2000] # all but the last 2000 words
simple_word_list = legal_word_list[:100] # first 100 words as a starter pack that is given.

N = len(simple_word_list[0]) # number of characters in each word
start = time.time
vocabulary = MySortedList() # It can also be MyBST or MySortedList or MyHashMap
# all of these classes must support an insert function and a search function
for el in legal_word_list:
    vocabulary.insert(el) 
wordle_answers = []
for j in range(20): # 20 puzzles per tournament
    wordle_answers.append(legal_word_list[random.randint(0,len(legal_word_list)-1)])
    wordle_answers.append(simple_word_list[random.randint(0,len(simple_word_list)-1)])

for j in range(20):
    correct_solution = wordle_answers[j]
    guess_list = []
    colors_list = []

    while 2>1: # In each cycle of this loop one valid guess is made and a new colors info is acquired.
        num_try = 0 # try number to guess a word in vocabulary that agrees with the results so far
        error_flag = False
        while 2>1: 
            # Within each cycle of this loop an attmept to find a word in the vocabulary that obeys colors is made.
            # the loop terminates when a valid word in the vocabulary is found
            guess =  guess_word_from_prev(guess_list, colors_list, simple_word_list, num_try)
            num_try += 1
            if vocabulary.search(guess):
                break
            if num_try > 10000:
                error_flag = True
                break
        if error_flag:
            break
        guess_list.append(guess)
        colors = give_colors(correct_solution, guess)
        colors_list.append(colors)
        if sum(colors) == 2*N:
            break
        if len(guess_list)>1000:
            error_flag = True
            break
    print("Tournament 1, puzzle ",j)
    if error_flag:
        print("Tried too many non-dictionary words in a single situation or tried too many guesses.")
    else:
        print("Guesses :", guess_list)
        print("Number of guesses :", len(guess_list))
        print("Solution :", correct_solution)
    print("====================")

end = time.time
print(f'total time taken is {round(end-start,4)} secs')